# Steam Games Dataset - Comprehensive Data Science Analysis

This notebook demonstrates a comprehensive data science analysis of the Steam Games dataset. As a data scientist, I'll showcase various techniques including:
- Data loading and exploration
- Data cleaning and preprocessing
- Exploratory Data Analysis (EDA)
- Statistical analysis
- Data visualization
- Insights and findings

## 1. Import Libraries and Load Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime
import ast

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("Libraries imported successfully!")

In [ ]:
# Load the dataset
df = pd.read_csv('steam_games.csv')
print(f"Dataset loaded successfully! Shape: {df.shape}")
print(f"Total games: {len(df):,}")

## 2. Initial Data Exploration

In [ ]:
# Display first few rows
print("First 5 rows of the dataset:")
df.head()

In [ ]:
# Dataset information
print("Dataset Information:")
df.info()

In [ ]:
# Statistical summary
print("Statistical Summary:")
df.describe()

In [ ]:
# Check for missing values
print("Missing Values:")
missing = df.isnull().sum()
missing_percent = (missing / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Percentage': missing_percent
})
missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False)

## 3. Data Cleaning and Preprocessing

In [ ]:
# Create a copy for analysis
df_clean = df.copy()

# Parse release_date and filter only released games for time-based analysis
df_clean['release_date_parsed'] = pd.to_datetime(df_clean['release_date'], errors='coerce')
df_clean['release_year'] = df_clean['release_date_parsed'].dt.year
df_clean['release_month'] = df_clean['release_date_parsed'].dt.month

# Parse list-like columns
def safe_eval(x):
    try:
        return ast.literal_eval(x) if isinstance(x, str) else []
    except:
        return []

df_clean['genres_list'] = df_clean['genres'].apply(safe_eval)
df_clean['categories_list'] = df_clean['categories'].apply(safe_eval)
df_clean['platforms_list'] = df_clean['platforms'].apply(safe_eval)

# Count of genres and categories per game
df_clean['genre_count'] = df_clean['genres_list'].apply(len)
df_clean['category_count'] = df_clean['categories_list'].apply(len)
df_clean['platform_count'] = df_clean['platforms_list'].apply(len)

print("Data preprocessing completed!")
print(f"Clean dataset shape: {df_clean.shape}")

## 4. Exploratory Data Analysis

### 4.1 Game Release Status

In [ ]:
# Release status
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Release status pie chart
release_status = df_clean['is_released'].value_counts()
axes[0].pie(release_status.values, labels=['Released', 'Not Released'], 
            autopct='%1.1f%%', startangle=90, colors=['#2ecc71', '#e74c3c'])
axes[0].set_title('Game Release Status', fontsize=14, fontweight='bold')

# Free vs Paid games
free_status = df_clean['is_free'].value_counts()
axes[1].pie(free_status.values, labels=['Paid', 'Free'], 
            autopct='%1.1f%%', startangle=90, colors=['#3498db', '#f39c12'])
axes[1].set_title('Free vs Paid Games', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print(f"Released games: {release_status[True]:,} ({release_status[True]/len(df_clean)*100:.1f}%)")
print(f"Free games: {free_status[True]:,} ({free_status[True]/len(df_clean)*100:.1f}%)")

### 4.2 Price Distribution Analysis

In [ ]:
# Price analysis for paid games
paid_games = df_clean[df_clean['price_initial (USD)'] > 0]['price_initial (USD)']

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Price distribution histogram
axes[0, 0].hist(paid_games, bins=50, color='skyblue', edgecolor='black', alpha=0.7)
axes[0, 0].set_xlabel('Price (USD)', fontweight='bold')
axes[0, 0].set_ylabel('Frequency', fontweight='bold')
axes[0, 0].set_title('Price Distribution (All Paid Games)', fontsize=12, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)

# Price distribution for games under $50
paid_games_under_50 = paid_games[paid_games <= 50]
axes[0, 1].hist(paid_games_under_50, bins=50, color='lightcoral', edgecolor='black', alpha=0.7)
axes[0, 1].set_xlabel('Price (USD)', fontweight='bold')
axes[0, 1].set_ylabel('Frequency', fontweight='bold')
axes[0, 1].set_title('Price Distribution (Games Under $50)', fontsize=12, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)

# Box plot for price
axes[1, 0].boxplot(paid_games_under_50, vert=False)
axes[1, 0].set_xlabel('Price (USD)', fontweight='bold')
axes[1, 0].set_title('Price Box Plot (Games Under $50)', fontsize=12, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)

# Price statistics
price_stats = f"""Price Statistics:
Mean: ${paid_games.mean():.2f}
Median: ${paid_games.median():.2f}
Mode: ${paid_games.mode()[0]:.2f}
Std Dev: ${paid_games.std():.2f}
Min: ${paid_games.min():.2f}
Max: ${paid_games.max():.2f}
25th %ile: ${paid_games.quantile(0.25):.2f}
75th %ile: ${paid_games.quantile(0.75):.2f}"""

axes[1, 1].text(0.1, 0.5, price_stats, fontsize=11, verticalalignment='center',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
axes[1, 1].axis('off')
axes[1, 1].set_title('Price Statistics Summary', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

### 4.3 Genre Analysis

In [ ]:
# Extract all genres and count them
all_genres = []
for genres in df_clean['genres_list']:
    all_genres.extend(genres)

genre_counts = pd.Series(all_genres).value_counts()

# Top 15 genres
fig, axes = plt.subplots(1, 2, figsize=(18, 6))

# Bar chart
top_15_genres = genre_counts.head(15)
axes[0].barh(range(len(top_15_genres)), top_15_genres.values, color='steelblue')
axes[0].set_yticks(range(len(top_15_genres)))
axes[0].set_yticklabels(top_15_genres.index)
axes[0].set_xlabel('Number of Games', fontweight='bold')
axes[0].set_title('Top 15 Most Popular Genres', fontsize=14, fontweight='bold')
axes[0].invert_yaxis()
axes[0].grid(True, alpha=0.3, axis='x')

# Add value labels
for i, v in enumerate(top_15_genres.values):
    axes[0].text(v, i, f' {v:,}', va='center')

# Pie chart for top 10
top_10_genres = genre_counts.head(10)
colors = plt.cm.Set3(range(len(top_10_genres)))
axes[1].pie(top_10_genres.values, labels=top_10_genres.index, autopct='%1.1f%%',
            startangle=90, colors=colors)
axes[1].set_title('Top 10 Genres Distribution', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print(f"Total unique genres: {len(genre_counts)}")
print(f"\nTop 5 genres:")
for i, (genre, count) in enumerate(genre_counts.head(5).items(), 1):
    print(f"{i}. {genre}: {count:,} games")

### 4.4 Platform Analysis

In [ ]:
# Platform availability
all_platforms = []
for platforms in df_clean['platforms_list']:
    all_platforms.extend(platforms)

platform_counts = pd.Series(all_platforms).value_counts()

fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Platform distribution
axes[0].bar(platform_counts.index, platform_counts.values, 
            color=['#3498db', '#2ecc71', '#e74c3c'])
axes[0].set_xlabel('Platform', fontweight='bold')
axes[0].set_ylabel('Number of Games', fontweight='bold')
axes[0].set_title('Games Available by Platform', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3, axis='y')

# Add value labels
for i, (platform, count) in enumerate(platform_counts.items()):
    axes[0].text(i, count, f'{count:,}', ha='center', va='bottom', fontweight='bold')

# Multi-platform analysis
platform_count_dist = df_clean['platform_count'].value_counts().sort_index()
axes[1].bar(platform_count_dist.index, platform_count_dist.values, color='coral')
axes[1].set_xlabel('Number of Platforms', fontweight='bold')
axes[1].set_ylabel('Number of Games', fontweight='bold')
axes[1].set_title('Games by Number of Supported Platforms', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='y')

# Add value labels
for i, v in enumerate(platform_count_dist.values):
    axes[1].text(platform_count_dist.index[i], v, f'{v:,}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

print("Platform Statistics:")
for platform, count in platform_counts.items():
    print(f"{platform.capitalize()}: {count:,} games ({count/len(df_clean)*100:.1f}%)")

### 4.5 Release Timeline Analysis

In [ ]:
# Games released per year
released_games = df_clean[df_clean['is_released'] == True].copy()
games_per_year = released_games['release_year'].value_counts().sort_index()

# Filter to reasonable years (e.g., 2000 onwards)
games_per_year_filtered = games_per_year[games_per_year.index >= 2000]

fig, axes = plt.subplots(2, 1, figsize=(15, 10))

# Line plot
axes[0].plot(games_per_year_filtered.index, games_per_year_filtered.values, 
             marker='o', linewidth=2, markersize=4, color='darkblue')
axes[0].fill_between(games_per_year_filtered.index, games_per_year_filtered.values, 
                      alpha=0.3, color='lightblue')
axes[0].set_xlabel('Year', fontweight='bold')
axes[0].set_ylabel('Number of Games Released', fontweight='bold')
axes[0].set_title('Steam Games Released Per Year (2000 onwards)', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Bar plot for recent years
recent_years = games_per_year_filtered[games_per_year_filtered.index >= 2015]
axes[1].bar(recent_years.index, recent_years.values, color='teal', alpha=0.7)
axes[1].set_xlabel('Year', fontweight='bold')
axes[1].set_ylabel('Number of Games Released', fontweight='bold')
axes[1].set_title('Games Released Per Year (2015 onwards)', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='y')

# Add value labels for recent years
for year, count in recent_years.items():
    axes[1].text(year, count, f'{count:,}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

print(f"Total released games with valid dates: {len(released_games[released_games['release_year'].notna()]):,}")
if len(recent_years) > 0:
    print(f"Peak year: {recent_years.idxmax()} with {recent_years.max():,} games")

### 4.6 Review Analysis

In [ ]:
# Filter games with reviews
games_with_reviews = df_clean[df_clean['total_reviews'] > 0].copy()

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Review count distribution
axes[0, 0].hist(np.log10(games_with_reviews['total_reviews'] + 1), 
                bins=50, color='mediumpurple', edgecolor='black', alpha=0.7)
axes[0, 0].set_xlabel('Log10(Review Count)', fontweight='bold')
axes[0, 0].set_ylabel('Frequency', fontweight='bold')
axes[0, 0].set_title('Distribution of Review Counts (Log Scale)', fontsize=12, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)

# Positive percentage distribution
axes[0, 1].hist(games_with_reviews['positive_percentual'], 
                bins=50, color='seagreen', edgecolor='black', alpha=0.7)
axes[0, 1].set_xlabel('Positive Review Percentage', fontweight='bold')
axes[0, 1].set_ylabel('Frequency', fontweight='bold')
axes[0, 1].set_title('Distribution of Positive Review Percentages', fontsize=12, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)

# Review score distribution
review_scores = games_with_reviews['review_score_desc'].value_counts()
axes[1, 0].barh(range(len(review_scores)), review_scores.values, color='indianred')
axes[1, 0].set_yticks(range(len(review_scores)))
axes[1, 0].set_yticklabels(review_scores.index)
axes[1, 0].set_xlabel('Number of Games', fontweight='bold')
axes[1, 0].set_title('Games by Review Score Category', fontsize=12, fontweight='bold')
axes[1, 0].invert_yaxis()
axes[1, 0].grid(True, alpha=0.3, axis='x')

# Add value labels
for i, v in enumerate(review_scores.values):
    axes[1, 0].text(v, i, f' {v:,}', va='center')

# Scatter: Reviews vs Positive Percentage
sample_size = min(5000, len(games_with_reviews))
sample_games = games_with_reviews.sample(sample_size)
axes[1, 1].scatter(np.log10(sample_games['total_reviews'] + 1), 
                   sample_games['positive_percentual'],
                   alpha=0.5, s=20, color='royalblue')
axes[1, 1].set_xlabel('Log10(Review Count)', fontweight='bold')
axes[1, 1].set_ylabel('Positive Review %', fontweight='bold')
axes[1, 1].set_title(f'Review Count vs Positive % (Sample: {sample_size:,})', 
                     fontsize=12, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Games with reviews: {len(games_with_reviews):,} ({len(games_with_reviews)/len(df_clean)*100:.1f}%)")
print(f"Average positive review percentage: {games_with_reviews['positive_percentual'].mean():.2f}%")
print(f"Median total reviews: {games_with_reviews['total_reviews'].median():.0f}")

### 4.7 Achievement Analysis

In [ ]:
# Games with achievements
games_with_achievements = df_clean[df_clean['n_achievements'] > 0]

fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Achievement distribution
axes[0].hist(games_with_achievements['n_achievements'], 
             bins=50, color='gold', edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Number of Achievements', fontweight='bold')
axes[0].set_ylabel('Frequency', fontweight='bold')
axes[0].set_title('Distribution of Achievements per Game', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Box plot
axes[1].boxplot(games_with_achievements['n_achievements'], vert=True)
axes[1].set_ylabel('Number of Achievements', fontweight='bold')
axes[1].set_title('Achievement Count Box Plot', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Games with achievements: {len(games_with_achievements):,} ({len(games_with_achievements)/len(df_clean)*100:.1f}%)")
print(f"Average achievements per game: {games_with_achievements['n_achievements'].mean():.2f}")
print(f"Median achievements: {games_with_achievements['n_achievements'].median():.0f}")
print(f"Max achievements: {games_with_achievements['n_achievements'].max():.0f}")

### 4.8 Category Analysis

In [ ]:
# Extract all categories
all_categories = []
for categories in df_clean['categories_list']:
    all_categories.extend(categories)

category_counts = pd.Series(all_categories).value_counts()

# Top 20 categories
plt.figure(figsize=(12, 8))
top_20_categories = category_counts.head(20)
plt.barh(range(len(top_20_categories)), top_20_categories.values, color='orchid')
plt.yticks(range(len(top_20_categories)), top_20_categories.index)
plt.xlabel('Number of Games', fontweight='bold')
plt.title('Top 20 Most Common Game Categories', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(True, alpha=0.3, axis='x')

# Add value labels
for i, v in enumerate(top_20_categories.values):
    plt.text(v, i, f' {v:,}', va='center')

plt.tight_layout()
plt.show()

print(f"Total unique categories: {len(category_counts)}")
print(f"\nTop 5 categories:")
for i, (category, count) in enumerate(category_counts.head(5).items(), 1):
    print(f"{i}. {category}: {count:,} games")

### 4.9 Developer and Publisher Analysis

In [ ]:
# Top developers and publishers
top_developers = df_clean['developers'].value_counts().head(15)
top_publishers = df_clean['publishers'].value_counts().head(15)

fig, axes = plt.subplots(1, 2, figsize=(18, 8))

# Top developers
axes[0].barh(range(len(top_developers)), top_developers.values, color='skyblue')
axes[0].set_yticks(range(len(top_developers)))
axes[0].set_yticklabels([str(dev)[:40] for dev in top_developers.index])
axes[0].set_xlabel('Number of Games', fontweight='bold')
axes[0].set_title('Top 15 Most Productive Developers', fontsize=14, fontweight='bold')
axes[0].invert_yaxis()
axes[0].grid(True, alpha=0.3, axis='x')

# Top publishers
axes[1].barh(range(len(top_publishers)), top_publishers.values, color='lightcoral')
axes[1].set_yticks(range(len(top_publishers)))
axes[1].set_yticklabels([str(pub)[:40] for pub in top_publishers.index])
axes[1].set_xlabel('Number of Games', fontweight='bold')
axes[1].set_title('Top 15 Publishers by Game Count', fontsize=14, fontweight='bold')
axes[1].invert_yaxis()
axes[1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

print(f"Total unique developers: {df_clean['developers'].nunique():,}")
print(f"Total unique publishers: {df_clean['publishers'].nunique():,}")

### 4.10 Price vs Reviews Analysis

In [ ]:
# Analyze relationship between price and reviews
paid_reviewed = df_clean[(df_clean['price_initial (USD)'] > 0) & 
                         (df_clean['total_reviews'] > 0)].copy()

# Create price bins
paid_reviewed['price_bin'] = pd.cut(paid_reviewed['price_initial (USD)'], 
                                     bins=[0, 5, 10, 20, 30, 50, 100],
                                     labels=['$0-5', '$5-10', '$10-20', '$20-30', '$30-50', '$50+'])

# Average positive percentage by price bin
avg_positive_by_price = paid_reviewed.groupby('price_bin')['positive_percentual'].mean()

fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Bar chart
axes[0].bar(range(len(avg_positive_by_price)), avg_positive_by_price.values, 
            color='mediumseagreen', alpha=0.7)
axes[0].set_xticks(range(len(avg_positive_by_price)))
axes[0].set_xticklabels(avg_positive_by_price.index, rotation=45)
axes[0].set_ylabel('Average Positive Review %', fontweight='bold')
axes[0].set_xlabel('Price Range', fontweight='bold')
axes[0].set_title('Average Positive Reviews by Price Range', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3, axis='y')

# Add value labels
for i, v in enumerate(avg_positive_by_price.values):
    axes[0].text(i, v, f'{v:.1f}%', ha='center', va='bottom', fontweight='bold')

# Game count by price bin
game_count_by_price = paid_reviewed['price_bin'].value_counts().sort_index()
axes[1].bar(range(len(game_count_by_price)), game_count_by_price.values, 
            color='steelblue', alpha=0.7)
axes[1].set_xticks(range(len(game_count_by_price)))
axes[1].set_xticklabels(game_count_by_price.index, rotation=45)
axes[1].set_ylabel('Number of Games', fontweight='bold')
axes[1].set_xlabel('Price Range', fontweight='bold')
axes[1].set_title('Game Distribution by Price Range', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='y')

# Add value labels
for i, v in enumerate(game_count_by_price.values):
    axes[1].text(i, v, f'{v:,}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

## 5. Statistical Insights and Correlations

In [ ]:
# Select numerical columns for correlation analysis
numerical_cols = ['price_initial (USD)', 'total_reviews', 'total_positive', 'total_negative',
                  'review_score', 'positive_percentual', 'n_achievements', 'required_age',
                  'genre_count', 'category_count', 'platform_count']

# Calculate correlation matrix
corr_matrix = df_clean[numerical_cols].corr()

# Plot correlation heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Correlation Matrix of Numerical Features', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

# Print strongest correlations
print("Strongest correlations (excluding self-correlations):")
corr_pairs = []
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        corr_pairs.append((
            corr_matrix.columns[i],
            corr_matrix.columns[j],
            corr_matrix.iloc[i, j]
        ))

corr_pairs.sort(key=lambda x: abs(x[2]), reverse=True)
for i, (col1, col2, corr) in enumerate(corr_pairs[:10], 1):
    print(f"{i}. {col1} <-> {col2}: {corr:.3f}")

## 6. Advanced Insights

In [ ]:
# Genre vs Price Analysis
genre_price_data = []
for idx, row in df_clean[df_clean['price_initial (USD)'] > 0].iterrows():
    for genre in row['genres_list']:
        genre_price_data.append({
            'genre': genre,
            'price': row['price_initial (USD)']
        })

genre_price_df = pd.DataFrame(genre_price_data)
avg_price_by_genre = genre_price_df.groupby('genre')['price'].mean().sort_values(ascending=False).head(15)

plt.figure(figsize=(12, 6))
plt.barh(range(len(avg_price_by_genre)), avg_price_by_genre.values, color='darkorange')
plt.yticks(range(len(avg_price_by_genre)), avg_price_by_genre.index)
plt.xlabel('Average Price (USD)', fontweight='bold')
plt.title('Top 15 Genres by Average Game Price', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(True, alpha=0.3, axis='x')

# Add value labels
for i, v in enumerate(avg_price_by_genre.values):
    plt.text(v, i, f' ${v:.2f}', va='center')

plt.tight_layout()
plt.show()

In [ ]:
# Most reviewed games
most_reviewed = df_clean.nlargest(20, 'total_reviews')[['name', 'total_reviews', 'positive_percentual', 'price_initial (USD)']]

print("Top 20 Most Reviewed Games:")
print("=" * 100)
for idx, (i, row) in enumerate(most_reviewed.iterrows(), 1):
    print(f"{idx}. {row['name'][:50]:50s} | Reviews: {row['total_reviews']:>8,} | "
          f"Positive: {row['positive_percentual']:>5.1f}% | Price: ${row['price_initial (USD)']:>6.2f}")

In [ ]:
# Best rated games with significant reviews (>100 reviews)
significant_reviews = df_clean[df_clean['total_reviews'] > 100]
best_rated = significant_reviews.nlargest(20, 'positive_percentual')[['name', 'positive_percentual', 'total_reviews', 'price_initial (USD)']]

print("\nTop 20 Best Rated Games (with >100 reviews):")
print("=" * 100)
for idx, (i, row) in enumerate(best_rated.iterrows(), 1):
    print(f"{idx}. {row['name'][:50]:50s} | Positive: {row['positive_percentual']:>5.1f}% | "
          f"Reviews: {row['total_reviews']:>8,} | Price: ${row['price_initial (USD)']:>6.2f}")

## 7. Key Findings and Conclusions

In [ ]:
# Generate comprehensive summary
summary = f"""
╔══════════════════════════════════════════════════════════════════════════════╗
║                    STEAM GAMES DATASET - KEY FINDINGS                        ║
╚══════════════════════════════════════════════════════════════════════════════╝

📊 DATASET OVERVIEW
   • Total Games: {len(df_clean):,}
   • Released Games: {df_clean['is_released'].sum():,} ({df_clean['is_released'].sum()/len(df_clean)*100:.1f}%)
   • Free Games: {df_clean['is_free'].sum():,} ({df_clean['is_free'].sum()/len(df_clean)*100:.1f}%)

💰 PRICING INSIGHTS
   • Average Game Price: ${paid_games.mean():.2f}
   • Median Game Price: ${paid_games.median():.2f}
   • Most Common Price Point: ${paid_games.mode()[0]:.2f}
   • Price Range: ${paid_games.min():.2f} - ${paid_games.max():.2f}

🎮 GENRE & CATEGORY
   • Total Unique Genres: {len(genre_counts)}
   • Most Popular Genre: {genre_counts.index[0]} ({genre_counts.values[0]:,} games)
   • Total Unique Categories: {len(category_counts)}
   • Most Common Category: {category_counts.index[0]} ({category_counts.values[0]:,} games)

💻 PLATFORM DISTRIBUTION
   • Windows: {platform_counts.get('windows', 0):,} games
   • Mac: {platform_counts.get('mac', 0):,} games
   • Linux: {platform_counts.get('linux', 0):,} games

⭐ REVIEW METRICS
   • Games with Reviews: {len(games_with_reviews):,} ({len(games_with_reviews)/len(df_clean)*100:.1f}%)
   • Average Positive Review %: {games_with_reviews['positive_percentual'].mean():.2f}%
   • Median Total Reviews: {games_with_reviews['total_reviews'].median():.0f}

🏆 ACHIEVEMENTS
   • Games with Achievements: {len(games_with_achievements):,} ({len(games_with_achievements)/len(df_clean)*100:.1f}%)
   • Average Achievements: {games_with_achievements['n_achievements'].mean():.2f}
   • Max Achievements: {games_with_achievements['n_achievements'].max():.0f}

👥 DEVELOPERS & PUBLISHERS
   • Unique Developers: {df_clean['developers'].nunique():,}
   • Unique Publishers: {df_clean['publishers'].nunique():,}
   • Most Productive Developer: {top_developers.index[0]} ({top_developers.values[0]} games)

📈 KEY INSIGHTS:
   1. The Steam platform has experienced massive growth in game releases over the years
   2. Indie games dominate the platform, showing democratization of game development
   3. Most games are priced under $20, with a sweet spot around $5-$15
   4. Windows is the dominant platform, but multi-platform support is common
   5. User reviews show overall positive sentiment with an average of {games_with_reviews['positive_percentual'].mean():.1f}%
   6. Achievements are a popular feature, present in {len(games_with_achievements)/len(df_clean)*100:.1f}% of games
   7. There's a strong correlation between total reviews and positive/negative counts
   8. Free-to-play games represent a significant portion of the marketplace

╚══════════════════════════════════════════════════════════════════════════════╝
"""

print(summary)

## 8. Final Remarks

This analysis demonstrates comprehensive data science techniques including:

- **Data Loading & Exploration**: Understanding the dataset structure and characteristics
- **Data Cleaning**: Handling missing values, parsing complex data types
- **Feature Engineering**: Creating derived features for better analysis
- **Exploratory Data Analysis**: Using various visualization techniques
- **Statistical Analysis**: Computing correlations and distributions
- **Insight Generation**: Extracting meaningful business insights from data

The Steam games dataset provides rich opportunities for further analysis including:
- Predictive modeling for game success
- Genre recommendation systems
- Price optimization strategies
- Trend forecasting
- Sentiment analysis on reviews
- Market segmentation analysis